In [9]:
from transformers import AutoTokenizer, AutoModel
import torch.nn.functional as F

In [11]:
from torch import Tensor
def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

In [12]:
with open('../data/interim/VW_60330_German/0_10_text.txt', 'r') as file:
    data = file.read()

In [13]:
print(data)

Gegenüber der VW 60330: 2009-12 wurden folgende Änderungen vorgenommen: – Abschnitt 1 „Anwendungsbereich“ überarbeitet; – Abschnitt 2 „Symbole und Abkürzungen“ hinzugefügt; – Abschnitt 4.2.2 „Abisolieren“ überarbeitet; – Abschnitt 4.2.3 „Leiterende“ überarbeitet; – Abschnitt 4.3.4 „Abmessungen des Leitercrimps“ überarbeitet; – Abschnitt 4.3.7.3 „Asymmetrischer O-förmiger ELA-Crimp (Umschlingungscrimp)“ umbenannt und überarbeitet; – Bild 30 geändert; – Abschnitt 5.2 „Crimpvorrichtungen“ überarbeitet; – Abschnitt 5.3 „Crimpkraftüberwachung“ überarbeitet; – Bild 31 geändert; – Abschnitt 5.1.1 „Prüfumfang alternativer Crimpwerkzeuge“ überarbeitet; – Tabelle B.1 und Tabelle B.2 angepasst.


In [14]:
tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large')

In [15]:
model = AutoModel.from_pretrained('intfloat/multilingual-e5-large')

In [16]:
input_text = [f'passage: {data}']
print(input_text)

['passage: Gegenüber der VW 60330: 2009-12 wurden folgende Änderungen vorgenommen: – Abschnitt 1 „Anwendungsbereich“ überarbeitet; – Abschnitt 2 „Symbole und Abkürzungen“ hinzugefügt; – Abschnitt 4.2.2 „Abisolieren“ überarbeitet; – Abschnitt 4.2.3 „Leiterende“ überarbeitet; – Abschnitt 4.3.4 „Abmessungen des Leitercrimps“ überarbeitet; – Abschnitt 4.3.7.3 „Asymmetrischer O-förmiger ELA-Crimp (Umschlingungscrimp)“ umbenannt und überarbeitet; – Bild 30 geändert; – Abschnitt 5.2 „Crimpvorrichtungen“ überarbeitet; – Abschnitt 5.3 „Crimpkraftüberwachung“ überarbeitet; – Bild 31 geändert; – Abschnitt 5.1.1 „Prüfumfang alternativer Crimpwerkzeuge“ überarbeitet; – Tabelle B.1 und Tabelle B.2 angepasst.']


In [17]:
batch_dict = tokenizer(input_text, max_length=512, padding=True, truncation=True, return_tensors='pt')

In [18]:
batch_dict['input_ids'].shape

torch.Size([1, 210])

In [19]:
from chromadb import EmbeddingFunction, Embeddings

class MyEmbeddingFunction(EmbeddingFunction):
    def __init__(self, model_name: str):
        self.model = AutoModel.from_pretrained(model_name)

    def __call__(self, batch_dict: dict) -> Embeddings:        
        outputs = self.model(**batch_dict)
        
        embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
        embeddings = F.normalize(embeddings, p=2, dim=1)
        return embeddings.tolist()


In [20]:
import chromadb
import os

In [21]:
import re
def natural_keys(text):
    return [int(c) if c.isdigit() else c for c in re.split('(\d+)', text)]

def get_page_number(filename):
    match = re.match(r"(\d+)_", filename)
    return int(match.groups()[0]) if match else None

In [22]:
def read_file_content(filename):
    with open(filename, 'r') as file:
        return file.read()

In [23]:
def process_page(files, doc_name):
    combined_text = '\n'.join([read_file_content(file) for file in files])
    tensor_list = tokenize_and_chunk(combined_text, doc_name)
    return tensor_list

def tokenize_and_chunk(text, doc_name: str, max_length=512):
    # Tokenize the input text
    text_with_doc_name = doc_name + '\n' + text
    tokens = tokenizer.encode_plus(
        text_with_doc_name,
        add_special_tokens=True,
        max_length=max_length,
        truncation=False,
        return_tensors='pt'
    )
    
    # Check if the number of tokens exceeds the max_length
    if tokens['input_ids'].size(1) > max_length:
        # Find the position to split the text, avoiding splitting inside a word
        split_position = text.rfind('\n', 0, len(text) // 2)
        if split_position <= 0 or split_position >= len(text) // 2:
            split_position = text.rfind(' ', 0, len(text) // 2)
        
        # Split the text into two halves and recursively call this function
        first_half = tokenize_and_chunk(text[:split_position], doc_name, max_length)
        second_half = tokenize_and_chunk(text[split_position:], doc_name, max_length)
        
        # Combine the tensors from both halves
        return first_half + second_half
    else:
        # If the tokenized text is within the limit, return it as a list of one tensor
        return [(tokens, text_with_doc_name)]

In [24]:
from collections import defaultdict
for source_doc in os.listdir('../data/interim/'):
    files_by_page = defaultdict(list)
    for filename in os.listdir(f'../data/interim/{source_doc}'):
        page_number = get_page_number(filename)
        if page_number is not None:
            files_by_page[page_number].append(filename)
    for page_number in sorted(files_by_page):
        files_by_page[page_number].sort(key=natural_keys)
        tensor_pages = process_page([f'../data/interim/{source_doc}/{file}'
                                     for file in files_by_page[page_number]
                                     if file.endswith('text.txt')
                                     ], doc_name=source_doc)
        print(len(tensor_pages))
        break

1
1
1
1
1
1


In [26]:
page_number

0

In [39]:
client = chromadb.PersistentClient(path="../data/db/")

In [40]:
collection = client.create_collection(name="texts_e5",
                                      metadata={'hnsw:space': 'cosine'},
                                      get_or_create=True)

In [42]:
tensor_pages[0][1]

'© Schäfer Werkzeug- und Sondermaschinenbau GmbH | 5.1\nCRIMPMASCHINE EPS 2001\nHOCHQUALITATIVE ELEKTRISCHE CRIMPVERBINDUNGEN\nVerbinden von Leitungen mit Kontakten\nDie Maschine ist zum Crimpen von Einzeladern  oder mehradrigen Mantelleitungen an automatisch  zugeführte Kontakte konzipiert.\nDie zu verarbeitenden Leitungen werden vom  Bediener eingelegt und der Crimpvorgang  durch Betätigen des Fußschalters gestartet. Die  robuste Maschinenauslegung ermöglicht eine  hohe Haltbarkeit und die Reproduzierbarkeit der  Crimpverbindungen.\nDer modulare Aufbau des Crimpautomats steht   für kurze Rüstzeiten und zur individuellen  Erweiterung an kundenspezfiische Anforderungen.\nVielfältige Anwendungsmöglichkeiten\nEine hohe Flexibilität und vielfältige Anwendungen  werden durch das einfache Umrüsten auf ein  anderes zu verarbeitendes Produkt ermöglicht.\nZum Crimpen unterschiedlicher Kontakte lassen  sich Crimpwerkzeuge über die Schnellwechsel- einrichtung einfach austauschen.\nDie ideale Ver

In [27]:
outputs = model(**tensor_pages[0][0])

embeddings = average_pool(outputs.last_hidden_state, tensor_pages[0][0]['attention_mask'])
embeddings = F.normalize(embeddings, p=2, dim=1)

In [28]:
embeddings.tolist()

[[0.006409656722098589,
  -0.017395662143826485,
  -0.004772012121975422,
  -0.041052501648664474,
  0.03726604953408241,
  -0.027244457975029945,
  0.00776702631264925,
  0.0613354817032814,
  0.06751231849193573,
  -0.008027937263250351,
  0.024266758933663368,
  0.03758355975151062,
  -0.010550914332270622,
  0.019615858793258667,
  -0.044624168425798416,
  -0.0263779629021883,
  -0.025678692385554314,
  0.023183709010481834,
  0.007168227806687355,
  0.011228004470467567,
  0.020605258643627167,
  -0.015557637438178062,
  -0.032363854348659515,
  -0.024022063240408897,
  -0.006759353913366795,
  0.0004234570369590074,
  -0.032927256077528,
  -0.04706837981939316,
  0.001250509056262672,
  -0.021193990483880043,
  -0.028245603665709496,
  -0.006691472139209509,
  -0.029756907373666763,
  -0.03504851460456848,
  0.015242272987961769,
  0.020179590210318565,
  0.04061463847756386,
  0.04255976155400276,
  -0.043746042996644974,
  0.016504306346178055,
  -0.015792714431881905,
  0.0090

In [31]:
import numpy as np

In [32]:
np.linalg.norm(embeddings.tolist()[0])

1.0000000467677517

In [53]:
collection.add(
    embeddings=embeddings.tolist(),
    documents=[tensor_pages[0][1]],
    metadatas=[{"page": 0, "document_name": source_doc}],
    ids=['0'],
)

TypeError: Collection.add() got an unexpected keyword argument 'data'

In [52]:
collection.peek()

{'ids': ['0'],
 'embeddings': [[0.001082798233255744,
   -0.016910143196582794,
   0.009277033619582653,
   -0.03573288023471832,
   0.06064322590827942,
   -0.02531571313738823,
   0.01625065878033638,
   0.07206086814403534,
   0.05145081505179405,
   -0.007944702170789242,
   0.02149966172873974,
   0.04368329793214798,
   -0.024645423516631126,
   0.0018145040376111865,
   -0.03756430745124817,
   -0.018718969076871872,
   -0.013118251226842403,
   0.03643091395497322,
   0.027043083682656288,
   -0.03212868049740791,
   0.039929669350385666,
   -0.007212747819721699,
   -0.015982715412974358,
   -0.0327148474752903,
   -0.036285918205976486,
   0.009878895245492458,
   -0.03394584730267525,
   -0.01207548938691616,
   -0.02031518518924713,
   -0.05707651749253273,
   -0.027246994897723198,
   0.003408594988286495,
   -0.020895857363939285,
   -0.0347662977874279,
   -0.007265743799507618,
   0.022721389308571815,
   0.03151383623480797,
   0.03484266996383667,
   -0.03218298032879

In [29]:
files_by_page

defaultdict(list,
            {14: ['14_0_text.txt',
              '14_1_text.txt',
              '14_2_text.txt',
              '14_3_text.txt',
              '14_4_text.txt',
              '14_5_text.txt',
              '14_6_text.txt',
              '14_7_text.txt',
              '14_8_text.txt',
              '14_9_text.txt',
              '14_10_text.txt',
              '14_11_text.txt',
              '14_12_text.txt'],
             7: ['7_0_text.txt',
              '7_1_text.txt',
              '7_2_text.txt',
              '7_3_text.txt',
              '7_4_text.txt',
              '7_5_text.txt',
              '7_6_text.txt',
              '7_7_text.txt',
              '7_8_text.txt',
              '7_9_text.txt',
              '7_10_text.txt',
              '7_11_text.txt',
              '7_12_text.txt',
              '7_13_text.txt',
              '7_14_text.txt',
              '7_15_text.txt'],
             12: ['12_0_text.txt',
              '12_1_text.txt',
              '

In [32]:
def process_page(files):
    combined_text = '\n'.join([read_file_content(file) for file in files])
    tensor_list = tokenize_and_chunk(combined_text)
    return tensor_list

In [ ]:
tensor_pages = [process_page(page_files) for page_files in list_of_lists]